In [1]:
import pickle
from pathlib import Path

import joblib
import keras
import matplotlib.pyplot as plt
import polars as pl
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

from utils import *

2024-04-02 04:39:22.026486: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 04:39:22.026593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 04:39:22.060792: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-02 04:39:22.129697: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
pc = ProjectConfig()
MODEL_DIR = pc.project_root_dir.joinpath("src").joinpath("model_fine_tuned_saves")
MODEL_NAME = "efficientnet_v2m"
MODEL_PATH = MODEL_DIR.joinpath(MODEL_NAME + ".h5")
CLASS_LIST_PATH = pc.project_root_dir.joinpath("src").joinpath("class_list.lzma")
TRAIN_DATA = pc.data_root_dir.joinpath("training_data")

No configuration file found. Using defaults.


In [6]:
if CLASS_LIST_PATH.exists():
    classes = joblib.load(CLASS_LIST_PATH)
else:
    print("No class list found, please run the training script first")

In [7]:
if TRAIN_DATA.exists() and TRAIN_DATA.is_dir():
    print("Test data found, loading")
    train_data = tf.keras.utils.image_dataset_from_directory(
        TRAIN_DATA,
        labels="inferred",
        label_mode="int",
        batch_size=32,
        image_size=(224, 224),
        shuffle=False,
    )
else:
    print("No test data found, where did it go?")

Test data found, loading
Found 31443 files belonging to 112 classes.


In [8]:
if MODEL_PATH.exists():
    model = tf.keras.models.load_model(MODEL_PATH)
    model.trainable = False
    print("Model loaded from disk")
else:
    print("Model not found at path:", MODEL_PATH)

Model loaded from disk


In [9]:
training_predictions_file = pc.data_root_dir.joinpath("training_predictions.lzma")
if training_predictions_file.exists():
    training_predictions = joblib.load(training_predictions_file)
else:
    predictions = model.predict(train_data)
    joblib.dump(predictions, training_predictions_file)

2024-04-02 04:41:16.003209: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906


983/983 [==============================] - 91s 86ms/step


In [10]:
predicted_classes = np.argmax(predictions, axis=-1)

In [16]:
file_paths = train_data.file_paths
class_names = train_data.class_names
true_labels = []
for images, true_label in train_data:
    true_labels.extend(true_label.numpy())
true_class_labels = [class_names[i] for i in true_labels]
predicted_class_labels = [class_names[i] for i in predicted_classes]


In [18]:
df = pl.DataFrame(zip(file_paths, true_class_labels, predicted_class_labels), schema=[("file_path", pl.Utf8), ("true_class_labels", pl.Utf8), ("predicted_label", pl.Utf8)])
df

file_path,true_class_labels,predicted_label
str,str,str
"""/tf/notebooks/…","""active dry yea…","""meat"""
"""/tf/notebooks/…","""active dry yea…","""beef"""
"""/tf/notebooks/…","""active dry yea…","""active dry yea…"
"""/tf/notebooks/…","""active dry yea…","""beef"""
"""/tf/notebooks/…","""active dry yea…","""active dry yea…"
…,…,…
"""/tf/notebooks/…","""zucchini""","""peanut butter"""
"""/tf/notebooks/…","""zucchini""","""peanut butter"""
"""/tf/notebooks/…","""zucchini""","""peanut butter"""
